In [60]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import warnings

# Set the environment variable to avoid the memory leak warning on Windows with MKL
os.environ["OMP_NUM_THREADS"] = "1"
# Suppress specific UserWarning related to KMeans memory leak
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows with MKL")


In [61]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [62]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [63]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [64]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_data_scaled = StandardScaler().fit_transform(
    df_market_data[["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", "price_change_percentage_30d",
                    "price_change_percentage_60d", "price_change_percentage_200d", "price_change_percentage_1y"]])

In [65]:
# Create a DataFrame with the scaled data
crypto_scaled_df = pd.DataFrame(
    crypto_data_scaled, columns=[
        "price_change_percentage_24h", "price_change_percentage_7d", 
        "price_change_percentage_14d", "price_change_percentage_30d", 
        "price_change_percentage_60d", "price_change_percentage_200d", 
        "price_change_percentage_1y"
    ]
)

# Copy the crypto names from the original data
crypto_scaled_df["coin_id"] = df_market_data.index

# Set the coin_id column as index (corrected)
crypto_scaled_df = crypto_scaled_df.set_index("coin_id")

# Display sample data
crypto_scaled_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Data.

In [66]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [67]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=42, n_init = 10)
    # 2. Fit the model to the data using df_market_data_scaled
    model.fit(crypto_scaled_df)
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(model.inertia_)


In [68]:
# Create a dictionary with the data to plot the Elbow curve

elbow_dict = {
    "k": k_values,
    "inertia": inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve

elbow_df = pd.DataFrame(elbow_dict)

In [69]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
curve1 = elbow_df.hvplot.line(
    x= 'k',
    y= 'inertia',
    width=800,
    height=400,
    rot=90,
    title = 'Elbow Curve',
    xticks = 'k')

curve1

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value of k is k = 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [70]:
# Initialize the K-Means model using the best value for k
best_k_model = KMeans(n_clusters= 4, random_state= 42, n_init= 10)

In [71]:
# Fit the K-Means model using the scaled data
best_k_model.fit(crypto_scaled_df)

KMeans(n_clusters=4, n_init=10, random_state=42)

In [72]:
# Predict the clusters to group the cryptocurrencies using the scaled data

clusters = best_k_model.predict(crypto_scaled_df)

# Print the resulting array of cluster values.
print(clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [73]:
# Create a copy of the DataFrame
copy_df = crypto_scaled_df.copy()

In [74]:
# Add a new column to the DataFrame with the predicted clusters
copy_df['Cluster'] = clusters

# Display sample data
copy_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,2
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,2
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,2
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,2


In [75]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot = copy_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols=["coin_id"], 
    title="Cryptocurrency Clusters",
    xlabel="Price Change Percentage (24h)",
    ylabel="Price Change Percentage (7d)"
)

scatter_plot

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [76]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [77]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

crypto_pca = pca_model.fit_transform(copy_df)

# View the first five rows of the DataFrame. 

print(crypto_pca[:5])

[[ 0.44890795 -1.24537573 -0.8506404 ]
 [ 0.49536709 -0.899823   -1.31755943]
 [-0.81884571  0.07189909  0.69501507]
 [-0.84035711  0.08005407  0.54435952]
 [ 0.81324004 -2.66952035 -1.64332113]]


In [78]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
exp_var = pca_model.explained_variance_ratio_

print("Explained Variance Ratio:")
for i, ev in enumerate(exp_var):
    print(f"Principal Component {i+1}: {ev:.2f}")


Explained Variance Ratio:
Principal Component 1: 0.35
Principal Component 2: 0.31
Principal Component 3: 0.23


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** .89

In [79]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(data = crypto_pca,
                      columns=['Principal Component 1', 'Principal Component 2', 'Principal Component 3'])

# Copy the crypto names from the original data
pca_df['coin_id'] = df_market_data.index

# Set the coinid column as index
pca_df.set_index('coin_id', inplace=True)

# Display sample data
pca_df.head(10)

,Principal Component 1,Principal Component 2,Principal Component 3
coin_id,,,
bitcoin,0.448908,-1.245376,-0.850640
ethereum,0.495367,-0.899823,-1.317559
tether,-0.818846,0.071899,0.695015
ripple,-0.840357,0.080054,0.544360
bitcoin-cash,0.813240,-2.669520,-1.643321
binancecoin,0.822871,-1.682096,-0.905203
chainlink,0.913868,-1.128684,-2.730038
cardano,0.811102,-1.113378,-1.598655
litecoin,0.258728,-0.946884,-1.103560


---

### Find the Best Value for k Using the PCA Data

In [80]:
# Create a list with the number of k-values from 1 to 11
# No need, k_values already previously defined as such

In [81]:
# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model3 = KMeans(n_clusters=k, random_state=42, n_init = 10)
# 2. Fit the model to the data using `df_market_data_pca`
    model3.fit(pca_df)
# 3. Append the model.inertia_ to the inertia list
    inertia_values_pca.append(model3.inertia_)


In [82]:
# Create a dictionary with the data to plot the Elbow curve

elbow_dict_pca = {
    "k": k_values,
    "inertia": inertia_values_pca
}

# Create a DataFrame with the data to plot the Elbow curve

elbow_df_pca = pd.DataFrame(elbow_dict_pca)

In [83]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
curve2 = elbow_df_pca.hvplot.line(
    x= 'k',
    y= 'inertia',
    width=800,
    height=400,
    rot=90,
    title = 'Elbow Curve',
    xticks = 'k'
)

curve2

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k = 4 


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [84]:
# Initialize the K-Means model using the best value for k
# Since we have the same k, and still have best_k_model, we should be able to just fit it to the PCA data

In [85]:
# Fit the K-Means model using the PCA data
best_k_model.fit(pca_df)

KMeans(n_clusters=4, n_init=10, random_state=42)

In [86]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_pca = best_k_model.predict(pca_df)

# Print the resulting array of cluster values.
print(clusters_pca)

[3 3 0 0 3 3 3 3 3 0 0 0 0 3 0 3 0 0 3 0 0 3 0 0 0 0 0 0 3 0 0 0 2 3 0 0 1
 0 0 0 0]


In [87]:
# Create a copy of the DataFrame with the PCA data
copy_pca_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
copy_pca_df['Cluster'] = clusters_pca

# Display sample data
copy_pca_df.head(10)


,Principal Component 1,Principal Component 2,Principal Component 3,Cluster
coin_id,,,,
bitcoin,0.448908,-1.245376,-0.850640,3
ethereum,0.495367,-0.899823,-1.317559,3
tether,-0.818846,0.071899,0.695015,0
ripple,-0.840357,0.080054,0.544360,0
bitcoin-cash,0.813240,-2.669520,-1.643321,3
binancecoin,0.822871,-1.682096,-0.905203,3
chainlink,0.913868,-1.128684,-2.730038,3
cardano,0.811102,-1.113378,-1.598655,3
litecoin,0.258728,-0.946884,-1.103560,3


In [88]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot_pca = copy_pca_df.hvplot.scatter(
    x="Principal Component 1",
    y="Principal Component 2",
    by="Cluster",
    hover_cols=["coin_id"], 
    title="Cryptocurrency Clusters (using PCA)"
)

scatter_plot_pca

:NdOverlay   [Cluster]
   :Scatter   [Principal Component 1]   (Principal Component 2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [89]:
# Composite plot to contrast the Elbow curves

elbow_composite = curve1 * curve2
elbow_composite

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [91]:
# Composite plot to contrast the clusters
scatter_composite = (scatter_plot * scatter_plot_pca).opts(legend_position='bottom_right')
scatter_composite.opts(width=800, height=400, title= 'Composite Clusters')

:Overlay
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [Principal Component 1]   (Principal Component 2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features to cluster the data can lead to clearer clusters, and as such easier to interpret data. It also helps eliminate outliers, very clearly seen in this example. This helps the robustness of the model by making sure its focused on the most relevant data points. 